In [1]:
import requests
from bs4 import BeautifulSoup
import re
import MySQLdb
import dateutil.parser
import datetime

In [2]:
response = requests.get('http://www.covers.com/pageLoader/pageLoader.aspx?page=/data/nba/teams/teams.html')
page = response.text
teams = BeautifulSoup(page) 
links = teams.find('table').find_all('a')
teamlinks = {}
for link in links:
    teamlinks[link.text] = link['href']


In [3]:
print teamlinks

{u'Chicago': '/pageLoader/pageLoader.aspx?page=/data/nba/teams/team404198.html', u'Philadelphia': '/pageLoader/pageLoader.aspx?page=/data/nba/teams/team404083.html', u'Denver': '/pageLoader/pageLoader.aspx?page=/data/nba/teams/team404065.html', u'L.A. Clippers': '/pageLoader/pageLoader.aspx?page=/data/nba/teams/team404135.html', u'Indiana': '/pageLoader/pageLoader.aspx?page=/data/nba/teams/team404155.html', u'Golden State': '/pageLoader/pageLoader.aspx?page=/data/nba/teams/team404119.html', u'Phoenix': '/pageLoader/pageLoader.aspx?page=/data/nba/teams/team404029.html', u'Sacramento': '/pageLoader/pageLoader.aspx?page=/data/nba/teams/team403975.html', u'Dallas': '/pageLoader/pageLoader.aspx?page=/data/nba/teams/team404047.html', u'Utah': '/pageLoader/pageLoader.aspx?page=/data/nba/teams/team404031.html', u'Brooklyn': '/pageLoader/pageLoader.aspx?page=/data/nba/teams/team404117.html', u'New Orleans': '/pageLoader/pageLoader.aspx?page=/data/nba/teams/team404101.html', u'Atlanta': '/pageLo

In [15]:
conn = MySQLdb.connect(host='dev-db.cycgavlmx4or.us-east-1.rds.amazonaws.com', user='admin', passwd='Look4more', db='nba_data')
cursor = conn.cursor()

In [16]:
for i in teamlinks:
    team =  i
    teamid =  re.findall('\d+', teamlinks[i])[0]
    print team, teamid
    bet_info_query = team_bets(team, teamid)
#     print bet_info_query
    db_commit(bet_info_query)
conn.close()

Chicago 404198
Philadelphia 404083
Denver 404065
L.A. Clippers 404135
Indiana 404155
Golden State 404119
Phoenix 404029
Sacramento 403975
Dallas 404047
Utah 404031
Brooklyn 404117
New Orleans 404101
Atlanta 404085
Memphis 404049
San Antonio 404302
Toronto 404330
Houston 404137
Miami 404171
Orlando 404013
Detroit 404153
Minnesota 403995
L.A. Lakers 403977
Oklahoma City 404316
Charlotte 664421
Boston 404169
Washington 404067
New York 404288
Cleveland 404213
Milwaukee 404011
Portland 403993


In [11]:
def team_bets(team, teamid):
    response = requests.get('http://www.covers.com/pageLoader/pageLoader.aspx?page=/data/nba/teams/pastresults/2014-2015/team%s.html'% teamid)
    page = response.text
    soup = BeautifulSoup(page)     
    gamebetLogs=[]
    try:
        rows = soup.find_all('table')[1].find_all('tr', { 'class': 'datarow'})
    except: 
        rows = soup.find_all('table')[0].find_all('tr', { 'class': 'datarow'})
    for row in rows:
        date = str(row.find_all('td')[0].text).strip(' \n\r@')
        opponent =  str(row.find_all('td')[1].text)
        score =  str(row.find_all('td')[2].text)
        spread =  str(row.find_all('td')[4].text)
        overunder =  str(row.find_all('td')[5].text)
        scores = re.findall('\d+', score)
        
        date= dateutil.parser.parse(date)


        if'PK'in spread:
            spread = 0
        elif '-' in spread:
            spread =  int(re.findall('\d+', spread)[0])*(-1)
        else:
            spread =  int(re.findall('\d+', spread)[0])
        overunder = int(re.findall('\d+', overunder)[0])

        if '@' in opponent:
            home_score = int(scores[1])
            away_score = int(scores[0])
        else:
            home_score = int(scores[0])
            away_score = int(scores[1])        
        opponents = opponent.strip(' \n\r@')

        game = (teamid, str(team), str(date), opponents, home_score, away_score, spread, overunder)
        gamebetLogs.append(game)
    query = """INSERT INTO bet_info  VALUES """ 
    query += ",".join([str(game) for game in gamebetLogs])
    return query

In [5]:
def db_commit(query):
    try:
        cursor.execute(query+ ';')
        conn.commit()
    except:     
        conn.rollback()
        print "Game not added"

In [12]:
final = team_bets('Chicago', '404198')

In [13]:
print final

INSERT INTO bet_info  VALUES ('404198', 'Chicago', '2015-04-15 00:00:00', 'Atlanta', 91, 85, -5, 198),('404198', 'Chicago', '2015-04-13 00:00:00', 'Brooklyn', 86, 113, -2, 198),('404198', 'Chicago', '2015-04-11 00:00:00', 'Philadelphia', 114, 107, -15, 195),('404198', 'Chicago', '2015-04-09 00:00:00', 'Miami', 78, 89, 2, 192),('404198', 'Chicago', '2015-04-08 00:00:00', 'Orlando', 105, 103, -7, 191),('404198', 'Chicago', '2015-04-05 00:00:00', 'Cleveland', 99, 94, 8, 194),('404198', 'Chicago', '2015-04-03 00:00:00', 'Detroit', 88, 82, -9, 196),('404198', 'Chicago', '2015-04-01 00:00:00', 'Milwaukee', 95, 91, -5, 191),('404198', 'Chicago', '2015-03-28 00:00:00', 'New York', 111, 80, -17, 190),('404198', 'Chicago', '2015-03-25 00:00:00', 'Toronto', 103, 116, -3, 197),('404198', 'Chicago', '2015-03-23 00:00:00', 'Charlotte', 98, 86, -7, 191),('404198', 'Chicago', '2015-03-21 00:00:00', 'Detroit', 107, 91, -3, 192),('404198', 'Chicago', '2015-03-20 00:00:00', 'Toronto', 108, 92, -1, 198),(

In [20]:
    response = requests.get('http://www.covers.com/pageLoader/pageLoader.aspx?page=/data/nba/teams/pastresults/2014-2015/team404029.html')
    page = response.text
    soup = BeautifulSoup(page)     
    gamebetLogs=[]
    rows = soup.find_all('table')[0].find_all('tr', { 'class': 'datarow'})

In [24]:
    for row in rows:
        date = str(row.find_all('td')[0].text).strip(' \n\r@')
        print date
        opponent =  str(row.find_all('td')[1].text)
        print opponent
        score =  str(row.find_all('td')[2].text)
        print score
        spread =  str(row.find_all('td')[4].text)
        print spread
        overunder =  str(row.find_all('td')[5].text)
        print overunder
        scores = re.findall('\d+', score)
        print scores
                
        if'PK'in spread:
            spread = 0
        else:
            spread =  int(re.findall('\d+', spread)[0])
        print spread
        overunder = int(re.findall('\d+', overunder)[0])
        print overunder


04/14/15

L.A. Clippers


        L 
        101-112

        P 11

        O 207
['101', '112']
11
207
04/12/15

        @ San Antonio


        L 
        91-107

        W 17

        U 199.5
['91', '107']
17
199
04/10/15

        @ New Orleans


        L 
        75-90

        L 9

        U 194
['75', '90']
9
194
04/08/15

        @ Dallas


        L 
        104-107

        W 8.5

        O 205.5
['104', '107']
8
205
04/07/15

        @ Atlanta


        L 
        69-96

        L 7

        U 200.5
['69', '96']
7
200
04/04/15

Utah


        W 
        87-85

        W -1

        U 187
['87', '85']
1
187
04/02/15

        @ Golden State


        L 
        106-107

        W 12.5

        O 210
['106', '107']
12
210
03/30/15

        @ Portland


        L 
        86-109

        L 8.5

        U 199
['86', '109']
8
199
03/29/15

Oklahoma City


        L 
        97-109

        L -2

        U 211.5
['97', '109']
2
211
03/27/15

Portland


        L 
        81-87

   

In [12]:
soup.find_all('table')[0]

<table cellpadding="2" cellspacing="1" class="data">
<tr class="datahead">
<td width="11%">Date</td>
<td width="21%">Vs</td>
<td width="18%">Score</td>
<td width="27%">Type</td>
<td width="13%">PHI Line</td>
<td width="10%">O/U</td>
</tr>
<tr class="datarow">
<td>
        04/15/15</td>
<td>
<a href="/pageLoader/pageLoader.aspx?page=/data/nba/teams/team404171.html">Miami</a>
</td>
<td>
        L <a href="/pageLoader/pageLoader.aspx?page=/data/nba/results/2014-2015/boxscore883923.html">
        101-105</a></td>
<td>
        Regular Season</td>
<td>
        L -2</td>
<td>
        O 192.5</td>
</tr>
<tr class="datarow">
<td>
        04/13/15</td>
<td>
<a href="/pageLoader/pageLoader.aspx?page=/data/nba/teams/team404011.html">Milwaukee</a>
</td>
<td>
        L <a href="/pageLoader/pageLoader.aspx?page=/data/nba/results/2014-2015/boxscore883900.html">
        97-107</a></td>
<td>
        Regular Season</td>
<td>
        L 3.5</td>
<td>
        O 196</td>
</tr>
<tr class="datarow">
<td>
     

In [ ]:
    gamebetLogs=[]
    rows = soup.find_all('table')[1].find_all('tr', { 'class': 'datarow'})